Author: Alex Dawson-Elli  
Class: 524 Optimization  
Due: 2/9/2017

#### Problem 1) polyhedron modeling 
Find the A matrix and b vector to satisfy $Ax\leq b$ for shape (a) and shape (b) these solutions were found by inspection 

##### (a) The cube
A cube has 6 faces, so we know that $A \in \mathbb{R}^{6x3} $ Taking z to be up, y to the right and x out, the equation for the a plane in three dimensions is given as:

$$
a(x - x_0) + b(y - y_0) + c(z - z_0) = 0 
$$

Where $p_0 = [x_0,y_0,z_0]$ is a point on the plane and the normal vector to the plane is given as $ \vec{n} = [a,b,c]$.  solving this equation right face, and plugging in a point on the plane ( $p = [0,1,0]$) 

$$
\vec{n} = [0,1,0]  
$$

$$
a(x - x_0) + b(y - y_0) + c(z - z_0) = 0 
$$

$$
0(x - x_0) + 1(y - y_0) + 0(z - z_0) = 0  
$$

$$
1(y - 1) = 0 
$$

$$
y = 1
$$

This agrees with our intuition. Following the same logic 5 more times, and using the equation of the plane as a boundry for the inequality, the description of the polyhedron is determined to be:

$$
-1 \leq x \leq 1
$$

$$
-1 \leq y \leq 1
$$

$$
-1 \leq z \leq 1
$$

Transforming this into $Ax\leq b$ form, we arrive at:
$$Ax\leq b$$

$$
\begin{bmatrix}
     1 &  0 &  0 \\
    -1 &  0 &  0 \\
     0 &  1 &  0 \\
     0 & -1 &  0 \\
     0 &  0 &  1 \\
     0 &  0 & -1 \\
\end{bmatrix}
\begin{bmatrix}
     x \\
     y \\
     z \\
\end{bmatrix}
\leq 
\begin{bmatrix}
     1 \\
     1 \\
     1 \\
     1 \\
     1 \\
     1 \\
\end{bmatrix}
$$

#### (b) Regular octahedron

Lets start by formally determining the normal vector for several of 8 planes using the cross product of two vectors in the plane

In [50]:
#1
x = [1, 0, 0]
y = [0, 1, 0]
z = [0, 0, 1]

v1 = y - x
v2 = z - x

println(cross(v1,v2)')

#2
v1 =  z - x
v2 = -y - x

println(cross(v1,v2)')

#3

v1 =  z - -y
v2 = -x - -y
println(cross(v1,v2)')

#4
v1 = z - -x
v2 = y - -z
println(cross(v1,v2))


[1 1 1]
[1 -1 1]
[-1 -1 1]
[-1, -1, 1]


From this test, we can conclude that the normal vector to a Regular Octahedron is defined by the unit vectors of the octant it resides in. 


In [57]:
#determine the constant b for the equation of the plane
function detEqu(n,pt)
    return dot(n,pt)
end

print(detEqu([1,1,1],[1,0,0]))
print(detEqu([1,-1,1],[1,0,0]))
print(detEqu([1,-1,-1],[1,0,0]))

#all these are 1:




111

From this we can huristically determine A and b to be:


$$Ax\leq b$$

$$
\begin{bmatrix}
     1  &  1  &  1 \\
     1  &  -1 & 1 \\
     -1  &  1  &  1 \\
     -1  & -1  &  1 \\
     1  &  1  &  -1 \\
     1  &  -1  & -1 \\
    -1  &  1  & -1 \\
    -1  & -1  & -1 \\
\end{bmatrix}
\begin{bmatrix}
     x \\
     y \\
     z \\
\end{bmatrix}
\leq 
\begin{bmatrix}
     1 \\
     1 \\
     1 \\
     1 \\
     1 \\
     1 \\
     1 \\
     1 \\
\end{bmatrix}
$$


#### Problem 2) standard form with equality constraints:



##### (b1) solve initial formulation of the optimization problem

First, we will solve the problem as it's initially presented, so our answer can be used to compare and check for algebreic mistakes in the Standard form.


In [31]:
using JuMP, Clp

m = Model(solver = ClpSolver())

@variable(m, z₁)       
@variable(m, -1 <= z₂ <= 5)          
@variable(m, -1 <= z₃ <= 5)
@variable(m, -2 <= z₄ <= 2) 
@constraint(m, -z₁ + 6z₂ - z₃ + z₄ >= -3)       #constraint 1
@constraint(m,       7z₂      + z₄ ==  5)       #constraint 2
@constraint(m,             z₃ + z₄ <=  2)       #constraint 3     
@objective(m, Max, 3z₁ - z₂)          

status = solve(m)

println(status)
println("Objective value is: ", getobjectivevalue(m))

Optimal
Objective value is: 25.28571428571429


The model solved, and we now have a standard of comparison

#### a) Transform LP into standard form
The objective is to convert the optimization problem from it's existing form into the standard form:  

$$
\text{minimize     } c^Tx 
$$

$$
\text{subject to:  }  Ax = b
$$

$$
x \geq 0
$$


This requires a fair bit of algebra, and the application of substitution rules or "Transformation Tricks" outlined in slides 3-21 to 3-22.  

**general approach**  
    all variables in the initial formulation were were written as $z_i$ , all transformed variables will be denoted $x_i$ regardless of if they are slack variable, variable substitutions or what have you. 
    
**place a bound on $z_1$**  
In the original model, $z_1 \in \mathbb{R}$. We transform this by adding:
$$z_1 = x_1 - x_2$$
$$ x_1 \geq 0 $$ 
$$x_2 \geq 0 $$
In accordance with rule 5.  

**transforming double sided inequalities**  
next, we transformed all inequalities of the form: 
$$min \leq z_i \leq max$$
into one constraint of the form:
$$x_i \geq 0$$
and the RHS of the inequality to an equality by adding a slack variable, as shown below:
$$
x_i - max \leq 0
$$

$$
x_i - max + x_{i+1} = 0  \quad ; \quad x_{i+1} \geq 0
$$

through this process, our three box constraints:
$$-1 \leq z_2 \leq 5 $$
$$-1 \leq z_3 \leq 5 $$
$$-2 \leq z_4 \leq 2 $$

become:


$$x_3 \geq 0$$
$$x_4 \geq 0$$
$$x_5 \geq 0$$

and:

$$ x_3 - 6 + x_6 = 0 \quad ; \quad x_6 \geq 0 $$
$$ x_4 - 6 + x_7 = 0 \quad ; \quad x_7 \geq 0 $$
$$ x_5 - 4 + x_8 = 0 \quad ; \quad x_8 \geq 0 $$

**convert constraint 1-3 into $x_i$ variable space, and remove inequalities**

beginning with constraint 1, lets multiply by -1 to flip arround the $\geq$ sign:
$$z_1 - 6z_2 + z_3 - z_4 - 3 \leq 0$$

applying the direct substitutions established earlier, we arrive at:
$$x_1 - x_2 - 6x_3 + x_4 - x_5 + 4 \leq 0 $$

applying rule 4 to add a slack variable:
$$x_1 - x_2 - 6x_3 + x_4 - x_5 + x_9 + 4  = 0  \quad ; \quad  x_9 \geq 0$$

constraint 2 is already an equality and through substitution it becomes:
$$7x_3 + x_5 = 14$$

adding a slack variable and substituting for 3 yeilds:
$$x_4 + x_5 + x_{10} = 5 \quad ; \quad x_{10} \geq 0$$


All equations are expressed in their matrix from in the summary section:

**conversion of objective function**
the objective is specified as a maximization. To turn it into a minimization, simply apply the negative


##### in summary:

**x domain varibles**
$$x_i \geq 0 \quad ∀_i ∈ \{1,2...10 \} $$

**direct substitutions:**
$$ z_1 = x_1 - x_2$$
$$ z_2 = x_3 - 1$$
$$ z_3 = x_4 -1$$
$$ z_4 = x_5 -2$$

**matrix equations**

$$
Ax = b
$$

$$ 
\begin{bmatrix}
     0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
     0 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 0 \\
     0 & 0 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 \\
     1 &-1 &-6 & 1 &-1 & 0 & 0 & 0 & 1 & 0 \\
     0 & 0 & 7 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
     0 & 0 & 0 & 1 & 1 & 0 & 0 & 0 & 0 & 1 \\
\end{bmatrix}
x
= 
\begin{bmatrix}
     6  \\
     6  \\
     4  \\
    -4  \\
     14 \\
     5  \\
\end{bmatrix}
$$

In [34]:
using JuMP, Clp

#define matricies
A = [0  0  1  0  0  1  0  0  0  0;
     0  0  0  1  0  0  1  0  0  0;
     0  0  0  0  1  0  0  1  0  0;
     1 -1 -6  1 -1  0  0  0  1  0;
     0  0  7  0  1  0  0  0  0  0;
     0  0  0  1  1  0  0  0  0  1;]

b = [6 6 4 -4 14 5]'

c = [-3 3 1 0 0 0 0 0 0 0]'

m = Model(solver = ClpSolver())


@variable(m, x[1:10] >= 0) 
@constraint(m, A*x .<= b)   
#@objective(m, Min, c'x[1] + 1)
@objective(m, Min, -3x[1] + 3x[2] + x[3]  -1)

status = solve(m)
println("objective = ", -getobjectivevalue(m) )



objective = 25.28571428571429


In [24]:
m

Feasibility problem with:
 * 6 linear constraints
 * 10 variables
Solver is ClpMathProg

#### problem 3) Alloy blending

A matrix approach was taken to this problem, which resulted in a compact system representation

In [18]:
#define model
m = Model(solver = ClpSolbver())
matNames = ["Iron Alloy 1", "Iron alloy 2", "Iron alloy 3", "Copper 1", "Copper 2", "Aluminum 1", "Aluminum 2"]

#define our system matricies
composition = [2.5  0.0  1.3;
               3.0  0.0  0.8;
               0.0  0.3  0.0;
               0.0  90   0.0;
               0.0  96   4.0;
               0.0  0.4  1.2;
               0.0  0.6  0.0 ]  * .01   #handle conversion from %

avail = [400,300,600,500,200,300,250]
cost  = [200,250,150,220,240,200,165]
minGrd = .01*[2.0,0.4,1.2]' #Minimum Grade
maxGrd = .01*[3.0,0.6,1.65]' #Max Grade


using JuMP, Clp

m = Model(solver=ClpSolver())
@variable(m, rawMaterials[1:7] >= 0 )  
@constraint(m, dot(rawMaterials,ones(7)) >= 500) 
@constraint(m, rawMaterials' .<= avail')                            
@constraint(m, minGrd * dot(rawMaterials,ones(7)) .<= rawMaterials'*composition) 
@constraint(m, rawMaterials'*composition .<= maxGrd * dot(rawMaterials,ones(7)))
@objective(m, Min, dot(rawMaterials,cost) )                                      

status = solve(m)
rm = []
#println(m)
for i = 1:7
    println(matNames[i], getvalue(rawMaterials[i]) )
    push!(rm,getvalue(rawMaterials[i]))
end
# println(status)
# println()
#println("p = ", getvalue() )
#println("q = ", getvalue() )
 println("objective = ", getobjectivevalue(m) )



Iron Alloy 1400.0
Iron alloy 20.0
Iron alloy 339.77630199231041
Copper 10.0
Copper 22.7612722824187346
Aluminum 157.46242572527083
Aluminum 20.0
objective = 98121.63579168123


#### problem 4) stingler's diet  

##### a) reproduce findings

Lets begin by importing the data from the CSV file - using the provided code. I've modified this code slightly to make implementing matrix constraints easier, as the notation is quite compact. 




In [24]:
# import Stigler's data set
raw = readcsv("stigler.csv")
(m,n) = size(raw)

n_nutrients = 2:n      # columns containing nutrients
n_foods = 3:m          # rows containing food names

nutrients = raw[1,n_nutrients][:]   # the list of nutrients (convert to 1-D array)
foods = raw[n_foods,1][:]           # the list of foods (convert to 1-D array)

# lower[i] is the minimum daily requirement of nutrient i.
lconst = convert(Array{Float64},raw[2,n_nutrients])  #named arrays are a bit verbose

# data[f,i] is the amount of nutrient i contained in food f.
dataMat = raw[n_foods,n_nutrients]  #named arrays are a bit verbose
dataMat = convert(Array{Float64},dataMat); #make it nice and floaty

In [25]:
using JuMP, Clp

m = Model(solver=ClpSolver())

@variable(m, Foods[1:size(foods)[1]] >= 0 )  
@constraint(m, Foods'*dataMat  .>= lconst')
@objective(m, Min, dot(Foods,ones(size(foods)[1])))                                      

status = solve(m)


:Optimal

Cool, our solver completed sucessfully. Lets check quickly that our constraints our met

In [245]:
foodsChoosen = []
for i = 1:77
    push!(foodsChoosen,getvalue(Foods[i]))    
end
println(foodsChoosen'*dataMat)
println(lconst')
differ = (convert(Array{Float64},foodsChoosen'*dataMat)[1,:] - lconst)'
println("diff = ", differ)

Any[3.0 147.414 0.8 60.4669 5.0 4.12044 2.7 27.316 75.0]
[3.0 70.0 0.8 12.0 5.0 1.8 2.7 18.0 75.0]
diff = [0.0 77.4135 1.11022e-16 48.4669 0.0 2.32044 0.0 9.31598 -1.42109e-14]


so we see our constraints are met. let's see how stigler did without a computer.

In [220]:
println("cost of food over a year is: ", getobjectivevalue(m)*365 )

cost of food over a year is: 39.66173154546625


Stigler didn't do too bad.

##### b) repeat with vegetarian diet

I've manually filtered the stigler list to only include vegetarian cuizine because let's face it, who has time to figure out how to pop a row from a Named Array in code. below, we reload the stiglerVeg.csv and repeat the above analysis


In [247]:
# import Stigler's data set
raw = readcsv("stiglerVeg.csv")
(m,n) = size(raw)

n_nutrients = 2:n      # columns containing nutrients
n_foods = 3:m          # rows containing food names

nutrients = raw[1,n_nutrients][:]   # the list of nutrients (convert to 1-D array)
foods = raw[n_foods,1][:]           # the list of foods (convert to 1-D array)

# lower[i] is the minimum daily requirement of nutrient i.
lconst = convert(Array{Float64},raw[2,n_nutrients])  #named arrays are a bit verbose

# data[f,i] is the amount of nutrient i contained in food f.
dataMat = raw[n_foods,n_nutrients]  #named arrays are a bit verbose
dataMat = convert(Array{Float64},dataMat); #make it nice and floaty

In [249]:
using JuMP, Clp

m = Model(solver=ClpSolver())

@variable(m, Foods[1:size(foods)[1]] >= 0 )  
@constraint(m, Foods'*dataMat  .>= lconst')
@objective(m, Min, dot(Foods,ones(size(foods)[1])))                                      

status = solve(m)


:Optimal

In [250]:
println("cost of food over a year is: ", getobjectivevalue(m)*365 )

cost of food over a year is: 39.79866435040896


The cost of food as a vegetarian is higher, but not by much